In [ ]:
import numpy as np

In [ ]:
def new_process(raw_data):
    transform_data = []
    for data in raw_data:
        trans_data = ntu_tranform_skeleton(data['input'])
        transform_data.append({'label': data['label'], 'input': trans_data})
    return transform_data

In [ ]:
def ntu_tranform_skeleton(test):
    """
    :param test: frames of skeleton within a video sample
    """
    remove_frame = False
    test = np.asarray(test)
    transform_test = []
    
    d = test[0,0:3]
    
    v1 = test[0,1*3:1*3+3]-test[0,0*3:0*3+3]
    v1 = v1/np.linalg.norm(v1)
    
    v2_ = test[0,12*3:12*3+3]-test[0,16*3:16*3+3]
    proj_v2_v1 = np.dot(v1.T,v2_)*v1/np.linalg.norm(v1)
    v2 = v2_-np.squeeze(proj_v2_v1)
    v2 = v2/np.linalg.norm(v2)
    
    v3 = np.cross(v2,v1)/np.linalg.norm(np.cross(v2,v1))
    
    v1 = np.reshape(v1,(3,1))
    v2 = np.reshape(v2,(3,1))
    v3 = np.reshape(v3,(3,1))
    
    R = np.hstack([v2,v3,v1])
    
    for i in range(test.shape[0]):
        xyzs = []
        for j in range(25):
            if test[i][j*3:j*3+3].all()==0:
                remove_frame = True
                break
            xyz = np.squeeze(np.matmul(np.linalg.inv(R),np.reshape(test[i][j*3:j*3+3]-d,(3,1))))
            xyzs.append(xyz)
        if not remove_frame:
            xyzs = np.reshape(np.asarray(xyzs),(-1,75))
            transform_test.append(xyzs)
        else:
            remove_frame = False
    
    transform_test = np.squeeze(np.asarray(transform_test))
    return transform_test.tolist()

## Example: Load data before View Invariant Transform

In [ ]:
import pickle
with open("/home/neuralnet/NTU_60/cross_subject_data/raw_train_data.pkl", "rb") as fin:
    train_crs = pickle.load(fin)
with open("/home/neuralnet/NTU_60/cross_subject_data/raw_test_data.pkl", "rb") as fin:
    test_crs = pickle.load(fin)

## Example: Transform and Save

In [ ]:
trans_train_crs = new_process(train_crs)
with open("/home/neuralnet/NTU_60/cross_subject_data/trans_train_data.pkl", "wb") as fout:
    pickle.dump(trans_train_crs, fout)
trans_test_crs = new_process(test_crs)
with open("/home/neuralnet/NTU_60/cross_subject_data/trans_test_data.pkl", "wb") as fout:
    pickle.dump(trans_test_crs, fout)